# Prepare test files

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.io import savemat
from scipy.ndimage.filters import uniform_filter

def angle_to_director(angle_8_bit):
    """
    Convert an 8-bit angle image to a director field.

    Args:
    angle_8_bit -- the input 8-bit angle image, [M x N].

    Returns:
    director -- the output director field, [M x N x 2].
    """
    angle = angle_8_bit * np.pi / 255 - np.pi / 2
    director = np.array([np.cos(angle), np.sin(angle)]).transpose(1, 2, 0)

    # require the y component director to be positive, otherwise flip the director
    director[director[:,:,1]<=0] *= -1

    return director
    
def qTensor(director, size=10):
    """
    Compute the Q-tensor from the director field.
    """
    Q = np.zeros((director.shape[0], director.shape[1], 2, 2))
    Q[:, :, 0, 0] = uniform_filter(director[:, :, 0] * director[:, :, 0], size=size) - 0.5
    Q[:, :, 0, 1] = uniform_filter(director[:, :, 0] * director[:, :, 1], size=size)
    Q[:, :, 1, 0] = uniform_filter(director[:, :, 1] * director[:, :, 0], size=size)
    Q[:, :, 1, 1] = uniform_filter(director[:, :, 1] * director[:, :, 1], size=size) - 0.5

    return Q

C:\Users\zl948\AppData\Local\Temp\ipykernel_34280\2127167446.py:5: DeprecationWarning: Please import `uniform_filter` from the `scipy.ndimage` namespace; the `scipy.ndimage.filters` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.filters import uniform_filter


In [15]:
director.shape

(500, 400)

In [ ]:
data = {}
folder = r"G:\My Drive\NASync\08 jun 2023\director_field"
for channel in ["A", "B", "C"]:
    angle = np.load(os.path.join(folder, f"07_{channel}.npy"))[300]
    director = angle_to_director(angle)
    q = qTensor(director, size=0)
    data[f"asym{channel}"] = q
    qs = qTensor(director, size=10)
    data[f"asym{channel}s"] = qs
folder = r"G:\My Drive\NASync\Sample 1\nd2\director_field"
for channel in ["A", "B", "C"]:
    angle = np.load(os.path.join(folder, f"06_{channel}.npy"))[12]
    director = angle_to_director(angle)
    q = qTensor(director, size=0)
    data[f"sym{channel}"] = q
    qs = qTensor(director, size=10)
    data[f"sym{channel}s"] = qs

In [32]:
savemat("director_fields.mat", data)

In [27]:
q.shape

(500, 400, 2, 2)